# Imports necessários


In [0]:
import tensorflow as tf

In [0]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [0]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

# Download do dataset do cifar10

In [0]:
(set_treino, set_validacao), dataset_info = tfds.load(
    'cifar10',
    split=['train[:70%]', 'train[70%:]'],
    with_info=True,
    as_supervised=True,
)

# Análise sobre o conjunto de dados

In [0]:
num_classes = dataset_info.features['label'].num_classes

num_treino_exemplos = 0
num_validacao_exemplos = 0

for exemplo in set_treino:
  num_treino_exemplos += 1

for exemplo in set_validacao:
  num_validacao_exemplos += 1

print('Numero total de classes: {}'.format(num_classes))
print('Numero total de imagens de treino: {}'.format(num_treino_exemplos))
print('Numero total de imagens de validação: {} \n'.format(num_validacao_exemplos))

Ajustando tamanho das imagens 

In [0]:
for i, exemplo in enumerate(set_treino.take(5)):
  print('Image {} shape: {} label: {}'.format(i+1, exemplo[0].shape, exemplo[1]))

#Transfer Learning da InceptionV3

Será feito o transfer learning utilizando o Tensorflow hub


In [0]:
IMAGE_RES = 299

def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32

batches_de_treino = set_treino.shuffle(num_treino_exemplos//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

batches_de_validacao= set_validacao.map(format_image).batch(BATCH_SIZE).prefetch(1)

(set_treino, set_validacao), dataset_info = tfds.load(
    'tf_flowers', 
    with_info=True, 
    as_supervised=True, 
    split=['train[:70%]', 'train[70%:]'],
)
batches_de_treino = set_treino.shuffle(num_treino_exemplos//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
batches_de_validacao = set_validacao.map(format_image).batch(BATCH_SIZE).prefetch(1)

URL = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)

model_inception = tf.keras.Sequential([
  feature_extractor,
  tf.keras.layers.Dense(num_classes)
])

model_inception.summary()

### Treinando o modelo

In [0]:
model_inception.compile(
  optimizer='adam', 
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 10

history = model_inception.fit(batches_de_treino,
                    epochs=EPOCHS,
                    validation_data=batches_de_validacao)